# LSTM на рівні символів у PyTorch

Мережа буде навчатися символ за символом на деякому тексті, а потім генерувати новий текст символ за символом. Як приклад, буде використано текст "Аліса в країні чудес". **Ця модель зможе генерувати новий текст на основі тексту з книги!**

Ця мережа базується на [пості про RNN Андрея Карпатого](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) та [реалізації на Torch](https://github.com/karpathy/char-rnn). Нижче наведена загальна архітектура символьно-орієнтованої RNN.

<img src="assets/charseq.jpeg" width="500">

Спочатку завантажимо необхідні ресурси для завантаження даних та створення моделі.

In [95]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

## Завантаження даних

Далі ми завантажимо текстовий файл "Аліса в країні чудес" і перетворимо його у цілі числа для використання в нашій мережі.

In [96]:
# open text file and read in data as `text`
with open('data/alice.txt', 'r', encoding='utf-8') as f:
    text = f.read()

Перевіримо перші 100 символів, щоб переконатися, що все гаразд.

In [97]:
text[:100]

'Alice’s Adventures in Wonderland\n\nby Lewis Carroll\n\nTHE MILLENNIUM FULCRUM EDITION 3.0\n\nContents\n\n C'

### Токенізація

У наступних клітинках створюються два **словники** для перетворення символів у цілі числа і навпаки. Кодування символів як цілих чисел полегшує їх використання як вхідних даних у мережі.

In [98]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

Можемо побачити ті ж символи, що й вище, закодовані як цілі числа.

In [99]:
encoded[:100]

array([80, 55, 72,  1, 82, 86, 17, 81, 80,  6, 22, 82, 47, 50, 11, 65, 82,
       17, 81, 72, 47, 81, 36,  4, 47,  6, 82, 65, 55,  2, 47,  6, 43, 43,
       21, 46, 81, 87, 82,  8, 72, 17, 81,  9,  2, 65, 65,  4, 55, 55, 43,
       43, 41, 62, 28, 81, 31, 57, 87, 87, 28, 60, 60, 57, 16, 31, 81, 48,
       16, 87,  9, 71, 16, 31, 81, 28, 35, 57, 41, 57, 34, 60, 81, 56, 83,
       64, 43, 43,  9,  4, 47, 50, 82, 47, 50, 17, 43, 43, 81,  9])

## Передобробка даних

Як видно на зображенні char-RNN вище, наш LSTM очікує на вхід **one-hot кодування**, тобто кожен символ спочатку перетворюється у ціле число (через створений словник), а потім у вектор-стовпець, де лише його індекс дорівнює 1, а решта — 0. Оскільки ми кодуємо дані у one-hot, давайте створимо функцію для цього!

In [100]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [101]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


## Створення міні-батчів для навчання

Для навчання на цих даних нам також потрібно створити міні-батчі. Пам'ятайте, що ми хочемо, щоб наші батчі складалися з кількох послідовностей певної довжини. Наприклад, наші батчі виглядатимуть так:

<img src="assets/sequence_batching@1x.png" width=500px>

<br>

У цьому прикладі ми беремо закодовані символи (передані як параметр `arr`) і розбиваємо їх на кілька послідовностей, визначених `batch_size`. Кожна послідовність матиме довжину `seq_length`.

### Створення батчів

**1. Спочатку потрібно відкинути частину тексту, щоб залишилися лише повні міні-батчі.**

Кожен батч містить $N \times M$ символів, де $N$ — розмір батчу (кількість послідовностей у батчі), а $M$ — довжина послідовності. Щоб отримати загальну кількість батчів $K$, які можна зробити з масиву `arr`, потрібно поділити довжину `arr` на кількість символів у батчі. Далі можна залишити лише $N * M * K$ символів.

**2. Далі потрібно розбити `arr` на $N$ батчів.**

Це можна зробити за допомогою `arr.reshape(size)`, де `size` — кортеж розмірів. Перший розмір — $N$, другий — `-1` (автоматично підбирається). Після цього отримаємо масив $N \times (M * K)$.

**3. Тепер можна ітеруватися по масиву, отримуючи міні-батчі.**

Кожен батч — це $N \times M$ вікно на масиві $N \times (M * K)$. Для кожного наступного батчу вікно зсувається на `seq_length`. Також потрібно створити масиви входів і цільових значень. Цільові значення — це вхідні, зсунуті на один символ вперед.

In [102]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
    '''
    n_batches = len(arr) // (batch_size * seq_length)
    arr = arr[:n_batches * batch_size * seq_length]
    arr = arr.reshape((batch_size, -1))
    for n in range(0, arr.shape[1], seq_length):
        x = arr[:, n:n+seq_length]
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

### Перевірте свою реалізацію

Тепер створимо деякі набори даних і подивимося, як формуються батчі. Наприклад, використаємо розмір батчу 8 і довжину послідовності 50.

In [103]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [104]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[80 55 72  1 82 86 17 81 80  6]
 [10  4 81 17 59 82 81 21 82 67]
 [ 4 11 55  6 42 81  2 47  6 81]
 [49 81 50 59 82 81 48  4  4 50]
 [17  2 72  6 42 81 70 57 50 81]
 [ 2 65 67 82 81 13 11 17 50  2]
 [ 4 81 53 43 82 47 47 46  8  4]
 [82  2 47 72 47 67 81 59 82 65]]

y
 [[55 72  1 82 86 17 81 80  6 22]
 [ 4 81 17 59 82 81 21 82 67  2]
 [11 55  6 42 81  2 47  6 81  8]
 [81 50 59 82 81 48  4  4 50 13]
 [ 2 72  6 42 81 70 57 50 81  8]
 [65 67 82 81 13 11 17 50  2 65]
 [81 53 43 82 47 47 46  8  4 65]
 [ 2 47 72 47 67 81 59 82 65 81]]


Якщо ви правильно реалізували `get_batches`, вивід вище має виглядати приблизно так:
```
x
 [[25  8 60 11 45 27 28 73  1  2]
 [17  7 20 73 45  8 60 45 73 60]
 ...
y
 [[ 8 60 11 45 27 28 73  1  2  2]
 [ 7 20 73 45  8 60 45 73 60 45]
 ...
 ```
 хоча точні числа можуть відрізнятися. Переконайтеся, що дані для `y` зсунуті на один крок вперед відносно `x`.

---
## Визначення мережі у PyTorch

Нижче ви визначите архітектуру мережі.

<img src="assets/charRNN.png" width=500px>

Далі, використовуючи PyTorch, визначимо архітектуру мережі. Спочатку визначаємо шари та операції. Потім — метод для прямого проходу. Також надано метод для передбачення символів.

In [105]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
device = "cpu"
if(train_on_gpu):
    print('Training on GPU!')
    device = "cuda"
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [106]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## define the layers of the model
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers,
                            dropout=drop_prob, batch_first=True)
        
        ## define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. '''
        lstm_out, hidden = self.lstm(x, hidden)
        out = self.dropout(lstm_out)
        out = out.contiguous().view(-1, self.n_hidden)
        out = self.fc(out)
        return out, hidden
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().to(device),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().to(device))
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        return hidden

## Час тренувати

Функція train дозволяє задати кількість епох, швидкість навчання та інші параметри.

Нижче використовується оптимізатор Adam та функція втрат крос-ентропії, оскільки вихід — це оцінки класів символів. Ми обчислюємо втрати та виконуємо зворотне поширення, як зазвичай!

Декілька деталей про навчання:
>* У циклі по батчах ми від'єднуємо прихований стан від історії; для LSTM це кортеж прихованого та cell-стану.
* Використовується [`clip_grad_norm_`](https://pytorch.org/docs/stable/_modules/torch/nn/utils/clip_grad.html) для запобігання вибуху градієнтів.

In [107]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10, early_stopping=None):
    net.train()
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.to(device)
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            if(train_on_gpu):
                inputs, targets = inputs.to(device), targets.to(device)
            h = tuple([each.data for each in h])
            net.zero_grad()
            output, h = net(inputs, h)
            loss = criterion(output, targets.view(batch_size*seq_length))
            loss.backward()
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # Calculate training accuracy
            preds = torch.argmax(output, dim=1)
            correct = (preds == targets.view(-1)).float().sum()
            accuracy = correct / (batch_size * seq_length)
            
            if counter % print_every == 0:
                val_h = net.init_hidden(batch_size)
                val_losses = []
                val_accuracies = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    val_h = tuple([each.data for each in val_h])
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.to(device), targets.to(device)
                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length))
                    val_losses.append(val_loss.item())
                    # Validation accuracy
                    preds = torch.argmax(output, dim=1)
                    correct = (preds == targets.view(-1)).float().sum()
                    val_acc = correct / (batch_size * seq_length)
                    val_accuracies.append(val_acc.item())
                net.train()
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Acc: {:.4f}...".format(accuracy.item()),
                      "Val Loss: {:.4f}...".format(np.mean(val_losses)),
                      "Val Acc: {:.4f}".format(np.mean(val_accuracies)))
                
                # Early stopping перевірка
                if early_stopping is not None:
                    if early_stopping(np.mean(val_losses)):
                        print(f"Early stopping at epoch {e+1}, step {counter}. Best val loss: {early_stopping.best_loss:.4f}")
                        return

## Ініціалізація моделі

Тепер можна тренувати мережу. Спочатку створимо саму мережу з заданими гіперпараметрами. Далі визначимо розміри міні-батчів і почнемо навчання!

In [108]:
## set your model hyperparameters
n_hidden = 512
n_layers = 2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(89, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=89, bias=True)
)


### Встановіть свої гіперпараметри для навчання!

In [109]:
batch_size = 128
seq_length = 100
n_epochs = 70 # start smaller if you are just testing initial behavior

In [110]:
# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/70... Step: 10... Loss: 3.3133... Acc: 0.0870... Val Loss: 3.4389... Val Acc: 0.1616
Epoch: 2/70... Step: 20... Loss: 3.2220... Acc: 0.1654... Val Loss: 3.4158... Val Acc: 0.1616
Epoch: 3/70... Step: 30... Loss: 3.1789... Acc: 0.1655... Val Loss: 3.3794... Val Acc: 0.1616
Epoch: 4/70... Step: 40... Loss: 3.1919... Acc: 0.1654... Val Loss: 3.3757... Val Acc: 0.1616
Epoch: 5/70... Step: 50... Loss: 3.2351... Acc: 0.1673... Val Loss: 3.3789... Val Acc: 0.1616
Epoch: 6/70... Step: 60... Loss: 3.1969... Acc: 0.1771... Val Loss: 3.3776... Val Acc: 0.1616
Epoch: 7/70... Step: 70... Loss: 3.1575... Acc: 0.1737... Val Loss: 3.3759... Val Acc: 0.1616
Epoch: 8/70... Step: 80... Loss: 3.1612... Acc: 0.1673... Val Loss: 3.3706... Val Acc: 0.1616
Epoch: 9/70... Step: 90... Loss: 3.1484... Acc: 0.1714... Val Loss: 3.3590... Val Acc: 0.1616
Epoch: 10/70... Step: 100... Loss: 3.1598... Acc: 0.1684... Val Loss: 3.3387... Val Acc: 0.1606
Epoch: 10/70... Step: 110... Loss: 3.0577... Acc: 0.1868..

## Отримання найкращої моделі

Щоб отримати найкращу продуктивність, слідкуйте за train та validation loss. Якщо train loss значно менший за validation loss — модель перенавчається. Збільшіть регуляризацію (dropout) або зменшіть мережу. Якщо train і validation loss близькі — модель недонавчається, збільшіть розмір мережі.

## Гіперпараметри

Ось гіперпараметри для мережі.

Для визначення моделі:
* `n_hidden` — кількість нейронів у прихованих шарах.
* `n_layers` — кількість прихованих LSTM-шарів.

Dropout та learning rate залишаємо за замовчуванням.

Для навчання:
* `batch_size` — кількість послідовностей у одному проході.
* `seq_length` — довжина послідовності символів, на якій навчається мережа. Більше — краще для довгих залежностей, але довше тренується. 100 — гарне значення.
* `lr` — швидкість навчання.

Поради від Андрея Карпатого щодо навчання мережі (оригінал [тут](https://github.com/karpathy/char-rnn#tips-and-tricks)):

> ## Поради та підказки

>### Відстеження validation loss та training loss
>Якщо ви новачок у ML чи нейромережах, важливо слідкувати за різницею між training loss (друкується під час навчання) та validation loss (друкується періодично на валідаційних даних).

> - Якщо training loss значно менший за validation loss — мережа перенавчається. Зменшіть розмір мережі або збільшіть dropout.
> - Якщо training/validation loss приблизно рівні — мережа недонавчається. Збільшіть розмір моделі (шари або кількість нейронів).

> ### Орієнтовна кількість параметрів

> Два найважливіші параметри — `n_hidden` та `n_layers`. Зазвичай використовують 2 або 3 шари. `n_hidden` підбирайте під розмір даних.

> - Кількість параметрів у моделі. Це друкується на початку навчання.
> - Розмір датасету. 1МБ ≈ 1 млн символів.

>Ці величини мають бути одного порядку.

> ### Стратегія для найкращих моделей

>Стратегія — робіть мережу максимально великою, на яку вистачає ресурсів, і підбирайте dropout. Вибирайте модель з найкращим validation loss.

>Зазвичай у deep learning запускають багато моделей з різними гіперпараметрами і беруть ту, що дала найкращий результат.

>Розмір train/validation split — теж параметр. Переконайтеся, що validation set достатньо великий, інакше результати будуть шумними.

## Збереження чекпойнту

Після навчання збережемо модель, щоб можна було завантажити її пізніше. Зберігаємо параметри для відтворення архітектури, гіперпараметри прихованого шару та символи тексту.

In [111]:
# change the name, for saving multiple files
model_name = 'rnn_x_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

---
## Генерація тексту

Після навчання моделі можна згенерувати текст. Для цього подаємо символ, мережа передбачає наступний символ, і так далі. Повторюючи це, отримаємо новий текст!

### Примітка щодо функції `predict`

Вихід RNN — це розподіл ймовірностей для наступного символу.

> Щоб отримати наступний символ, застосовуємо softmax, отримуємо ймовірності і вибираємо символ випадково згідно з ними.

### Top K sampling

Можна обмежити вибір лише $K$ найімовірнішими символами. Це зменшує випадковість і дозволяє уникнути абсурдних символів, але залишає елемент випадковості. Детальніше про [topk тут](https://pytorch.org/docs/stable/torch.html#torch.topk).

In [112]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.to(device)
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

### Прогрів та генерація тексту

Зазвичай потрібно "прогріти" мережу, щоб вона накопичила прихований стан. Інакше перші символи будуть випадковими, оскільки мережа ще не має історії.

In [113]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.to(device)
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [114]:
print(sample(net, 1000, prime='Alice', top_k=5))

Alice same to have out, and shakly a muck any read, ald that it saod off them,
say the way of her fuct of herself. “I was
next you can to
say
you she shand the madten on the time,” said the Caterpillar this latter wonsers when they a latter trough the distors, and she curled of the
rabes, and the
bard a trempline to be some
would would be a read-tiles and would began an time the large
till she was never a mentily have any must before sime to the was, the Queen surd thing it a very cone,, to she hastered the supteres with a look of thas
in a great
crosstente
to see the some of the chatcore, so me were next and bit has tried, “Wolling the courth!” as the Duca sily was tried, but the March Hare,
she could see in a moute olf other at all the carss in the tours, and a last of the withor, werl have off things at inst on tile a shere. “I won’t be, to tone,” And the King, and
Anciced as
it hould next seemed at hers lowe
time to sey west the came on the house, and so he said to the growing, but

## Завантаження чекпойнту

In [115]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open('rnn_x_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

C:\Users\kleot\AppData\Local\Temp\ipykernel_35876\161239107.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(f)


<All keys matched successfully>

In [116]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="card"))

card on the simpores to she lattle should hear the roust of
serpeat.

The Kang tarking. “I’m turning to that is I well so me out his a corsion, and I don’t knike it such so
shaming about the shease a grail herself that it meger thing of the triel, when
the Donmouse was so merembone the right of a grow tristing. “I’m a good off frenged to be shall,” said Alice, thinked to her even the wat anoush of the words and was as is, an the places
on this shariss. “What I can to tre to down that your wancon of to go any one head—‘Now the way off to be the firht
see how is, and she say
the sare they was no beas, that was the curtion, and whis to see hhas he chonges in the carsher; and Alice comled at her some, and was sige a little were sarish, and the Queen, say at ame all all the confurions and the rowfor, that the Duco,
sile
corsed to
he tunted to be angurden.

“Yis
a serm to to see theme all the fount at all,” soo Alice she hooked in ontended of the sard to see her heads the same, and the Moske